In [12]:
import argparse
import os
import sys
import logging
import pickle
from functools import partial
import time
from tqdm import tqdm
from collections import Counter
import random
import numpy as np

import torch
from torch.utils.data import DataLoader
import pytorch_lightning as pl
from pytorch_lightning import seed_everything
from pytorch_lightning.callbacks.early_stopping import EarlyStopping
from pytorch_lightning.callbacks.progress import TQDMProgressBar
from pytorch_lightning.callbacks import LearningRateMonitor


from torch.optim import AdamW
from transformers import T5ForConditionalGeneration, T5Tokenizer
from transformers import get_linear_schedule_with_warmup

from atoss.data_utils import *
from atoss.eval_utils import *
from atoss.process import *

torch.set_float32_matmul_precision('high')

In [13]:
# setting args
class Args:
    def __init__(self):
        self.path = '/home/elicer/ATOSS'
        self.method = 'sft' # task
        self.model = 'final'
        self.task = 'acos' # task
        self.dataset = 'rest16' # task 
        self.train = 'train' # task 
        self.dev = 'dev' # task 
        self.eval_data_split = 'test' # test or dev
        self.data_path = f'{self.path}/data/{self.method}/{self.task}/{self.dataset}'
        self.ctrl_token = "post"
        self.data_ratio = 1.0
        self.model_name_or_path = 't5-base' # used base model
        self.load_ckpt_name = 'epoch=9-val_f1=65.41-val_loss=0.14' # 사전 훈련된 모델의 체크포인트 파일로드 
        self.do_train = False # train or not
        self.do_inference = True # inference or not
        self.max_seq_length = 512 # 입력 시퀀스 최대 길이
        self.n_gpu = 1 # gpu 개수
        self.train_batch_size = 16
        self.eval_batch_size = 16
        self.gradient_accumulation_steps = 1
        self.learning_rate = 1e-5
        self.num_train_epochs = 20
        self.seed = 25
        self.weight_decay = 0.0
        self.adam_epsilon = 1e-8
        self.warmup_steps = 0.0
        self.multi_path = False
        self.num_path = 1
        self.beam_size = 1
        self.save_top_k = 1
        self.check_val_every_n_epoch = 10
        self.sort_label = False
        self.load_path_cache = False
        self.lowercase = True
        self.multi_task = False
        self.constrained_decode = False

def init_args():
    args = Args()

    args.output_dir =  f'{args.path}/outputs/{args.method}/{args.task}/{args.dataset}'

    # set up output dir which looks like './outputs/rest15/'
    if not os.path.exists(f'{args.path}/outputs'):
        os.mkdir(f'{args.path}/outputs')

    if not os.path.exists(args.output_dir):
        #os.mkdir(args.output_dir)
        os.makedirs(args.output_dir, exist_ok=True)

    return args

args = init_args()

print('method:', args.method)
print('data path:', args.data_path)
print('output path:', args.output_dir)

method: sft
data path: /home/elicer/ATOSS/data/sft/acos/rest16
output path: /home/elicer/ATOSS/outputs/sft/acos/rest16


In [14]:
set_seed(args.seed)

Random seed set as 25


In [15]:
class T5FineTuner(pl.LightningModule):
    """
    Fine tune a pre-trained T5 model
    """

    def __init__(self, config, tfm_model, tokenizer):
        super().__init__()
        self.save_hyperparameters(ignore=['tfm_model'])
        self.config = config
        self.model = tfm_model
        self.tokenizer = tokenizer

    def forward(self,
                input_ids,
                attention_mask=None,
                decoder_input_ids=None,
                decoder_attention_mask=None,
                labels=None):
        return self.model(
            input_ids,
            attention_mask=attention_mask,
            decoder_input_ids=decoder_input_ids,
            decoder_attention_mask=decoder_attention_mask,
            labels=labels,
        )

    def _step(self, batch):
        lm_labels = batch["target_ids"]
        lm_labels[lm_labels[:, :] == self.tokenizer.pad_token_id] = -100

        outputs = self(input_ids=batch["source_ids"],
                       attention_mask=batch["source_mask"],
                       labels=lm_labels,
                       decoder_attention_mask=batch['target_mask'])

        loss = outputs[0]
        return loss

    def training_step(self, batch, batch_idx):
        loss = self._step(batch)
        self.log("train_loss", loss)
        return loss

    def evaluate(self, batch, stage=None):
        # get f1
        outs = self.model.generate(input_ids=batch['source_ids'],
                                   attention_mask=batch['source_mask'],
                                   max_length=self.config.max_seq_length,
                                   return_dict_in_generate=True,
                                   output_scores=True,
                                   num_beams=1)

        dec = [
            self.tokenizer.decode(ids, skip_special_tokens=True)
            for ids in outs.sequences
        ]
        target = [
            self.tokenizer.decode(ids, skip_special_tokens=True)
            for ids in batch["target_ids"]
        ]
        scores, _, _ = compute_scores(dec, target, verbose=False)
        f1 = torch.tensor(scores['f1'], dtype=torch.float64)

        # get loss
        loss = self._step(batch)

        if stage:
            self.log(f"{stage}_loss",
                     loss,
                     prog_bar=True,
                     on_step=False,
                     on_epoch=True)
            self.log(f"{stage}_f1",
                     f1,
                     prog_bar=True,
                     on_step=False,
                     on_epoch=True)

    def validation_step(self, batch, batch_idx):
        self.evaluate(batch, "val")

    def test_step(self, batch, batch_idx):
        self.evaluate(batch, "test")

    def configure_optimizers(self):
        """ Prepare optimizer and schedule (linear warmup and decay) """
        model = self.model
        no_decay = ["bias", "LayerNorm.weight"]
        optimizer_grouped_parameters = [
            {
                "params": [
                    p for n, p in model.named_parameters()
                    if not any(nd in n for nd in no_decay)
                ],
                "weight_decay":
                self.config.weight_decay,
            },
            {
                "params": [
                    p for n, p in model.named_parameters()
                    if any(nd in n for nd in no_decay)
                ],
                "weight_decay":
                0.0,
            },
        ]
        optimizer = AdamW(optimizer_grouped_parameters,
                          lr=self.config.learning_rate,
                          eps=self.config.adam_epsilon)
        scheduler = {
            "scheduler":
            get_linear_schedule_with_warmup(optimizer,
                                            **self.config.lr_scheduler_init),
            "interval":
            "step",
        }
        return [optimizer], [scheduler]

    def train_dataloader(self):
        print("load training data.")
        train_dataset = ABSADataset(args=args,
                                    tokenizer=tokenizer,
                                    task_name=args.task,
                                    data_type=args.train,
                                    max_len=args.max_seq_length)
        dataloader = DataLoader(
            train_dataset,
            batch_size=self.config.train_batch_size,
            drop_last=True
            if args.data_ratio > 0.3 else False, # don't drop on few-shot
            shuffle=True,
            num_workers=2)

        return dataloader

    def val_dataloader(self):
        val_dataset = ABSADataset(args=args,
                                    tokenizer=tokenizer,
                                    task_name=args.task,
                                    data_type=args.dev,
                                    max_len=args.max_seq_length)
        return DataLoader(val_dataset,
                          batch_size=self.config.eval_batch_size,
                          num_workers=2)

    @staticmethod
    def rindex(_list, _value):
        return len(_list) - _list[::-1].index(_value) - 1


In [16]:
# do train
if args.do_train:

    # mvp sample 수 확인  
    print("\n", "=" * 30, f"NEW EXP: Task : {args.task}","=" * 30, "\n")
    tokenizer = T5Tokenizer.from_pretrained(args.model_name_or_path)
    # sanity check
    # show one sample to check the code and the expected output
    print(f"Here is an example (from the dev set):")
    dataset = ABSADataset(tokenizer=tokenizer,
                      task_name=args.task,
                      data_type=args.train,
                      args=args,
                      max_len=args.max_seq_length)

    # initialize the T5 model
    tfm_model = T5ForConditionalGeneration.from_pretrained(args.model_name_or_path)
    model = T5FineTuner(args, tfm_model, tokenizer)
    
    # load data
    train_loader = model.train_dataloader()
    
    # config optimizer
    t_total = ((len(train_loader.dataset) //
                (args.train_batch_size * max(1, args.n_gpu))) //
               args.gradient_accumulation_steps *
               float(args.num_train_epochs))
    
    args.lr_scheduler_init = {
        "num_warmup_steps": args.warmup_steps,
        "num_training_steps": t_total
    }
    
    checkpoint_callback = pl.callbacks.ModelCheckpoint(
        dirpath=args.output_dir,
        filename='{epoch}-{val_f1:.2f}-{val_loss:.2f}',
        monitor='val_f1',
        mode='max',
        save_top_k=args.save_top_k,
        save_last=False)
    
    early_stop_callback = EarlyStopping(monitor="val_f1",
                                        min_delta=0.00,
                                        patience=20,
                                        verbose=True,
                                        mode="max")
    lr_monitor = LearningRateMonitor(logging_interval='step')
    
    # prepare for trainer
    train_params = dict(
        accelerator="gpu",
        devices=1,
        default_root_dir=args.output_dir,
        accumulate_grad_batches=args.gradient_accumulation_steps,
        gradient_clip_val=1.0,
        max_epochs=args.num_train_epochs,
        check_val_every_n_epoch=args.check_val_every_n_epoch,
        callbacks=[
            checkpoint_callback, early_stop_callback,
            TQDMProgressBar(refresh_rate=10), lr_monitor
        ],
    )
    
    trainer = pl.Trainer(**train_params)
    
    trainer.fit(model)
    
    # save the final model
    model.model.save_pretrained(os.path.join(args.output_dir, args.model))
    tokenizer.save_pretrained(os.path.join(args.output_dir, args.model))
    print("Finish training and saving the model!")

In [17]:
# do inference
if args.do_inference:
    # load the outdir
    model_path = os.path.join(args.output_dir, args.model)
    print(model_path)
    tfm_model = T5ForConditionalGeneration.from_pretrained(model_path)
    tokenizer = T5Tokenizer.from_pretrained(model_path)
    model = T5FineTuner(args, tfm_model, tokenizer)
   
    print("\n****** Conduct inference on trained checkpoint ******")
    
    if args.load_ckpt_name:
        ckpt_path = os.path.join(args.output_dir, f'{args.load_ckpt_name}.ckpt')
        print("Loading ckpt:", ckpt_path)
        checkpoint = torch.load(ckpt_path)
        model.load_state_dict(checkpoint["state_dict"])
    
    log_file_path = os.path.join(args.output_dir, "result.txt")
    
    
    # compute the performance scores
    with open(log_file_path, "a+") as f:
        config_str = f" model: {args.model}, beam: {args.beam_size}, constrained: {args.constrained_decode}\n"
        print(config_str)
        f.write(config_str)
        scores = evaluate(args,
                          model,
                          args.task,
                          data_type=args.eval_data_split)
    
        exp_results = "model: {} data: {} precision: {:.2f} recall: {:.2f} F1 = {:.2f}".format(
            args.model, args.eval_data_split, scores['precision'], scores['recall'], scores['f1'])
        print(exp_results)
        f.write(exp_results + "\n")
        f.flush()

/home/elicer/ATOSS/outputs/sft/acos/rest16/final


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.



****** Conduct inference on trained checkpoint ******
Loading ckpt: /home/elicer/ATOSS/outputs/sft/acos/rest16/epoch=9-val_f1=65.41-val_loss=0.14.ckpt
 model: final, beam: 1, constrained: False

Total examples = 583
Total examples = 583


100%|██████████| 37/37 [00:56<00:00,  1.52s/it]

pred labels count Counter({1: 583})
gold  yum!
pred  yum!

gold  serves really good sushi.
pred  serves really good sushi.

gold  not the biggest portions but adequate.
pred  not the biggest portions but adequate.

gold  green tea creme brulee is a must!
pred  green tea creme brulee is a must!

gold  it has great sushi and even better service.
pred  it has great sushi. it has even better service.

gold  the entire staff was extremely accomodating and tended to my every need.
pred  the entire staff was extremely accomodating. they tended to my every need.

gold  i've been to this restaurant over a dozen times with no complaints to date.
pred  i've been to this restaurant over a dozen times with no complaints to date.

gold  the owner is belligerent to guests that have a complaint.
pred  the owner is belligerent to guests that have a complaint.

gold  good food!
pred  good food!

gold  this is a great place to get a delicious meal.
pred  this is a great place to get a delicious meal.

nu

In [18]:
file_path = os.path.join(
        args.output_dir, "result_{}_{}_{}_{}{}beam{}.pickle".format(
            args.method,
            args.model,
            args.eval_data_split,
            "best_" if args.load_ckpt_name else "",
            "cd_" if args.constrained_decode else "",
            args.beam_size))
with open(file_path, 'rb') as f:
    loaded_object = pd.read_pickle(f)
loaded_object[0][0]

'yum!'

In [19]:
file = f'/home/elicer/ABSA/data/{args.task}/{args.dataset}/test.txt'
_, targets = split_sharp(file)
targets[0]

Data read. Total count:  583


"[['NULL', 'food quality', 'positive', 'yum']]"

In [20]:
file = f'/home/elicer/ABSA/data/{args.task}/{args.dataset}/test.txt'
_, targets = split_sharp(file)
sft = merge_sharp_n(loaded_object[0],targets)
file_name = os.path.join(f'/home/elicer/ABSA/data/{args.task}/{args.dataset}',
                         "result_{}_{}_{}_{}{}beam{}.txt".format(
                             args.method,
                             args.model,
                             args.eval_data_split,
                              "best_" if args.load_ckpt_name else "",
                             "cd_" if args.constrained_decode else "",
                              args.beam_size))
print(file_name)
with open(file_name, 'w', encoding='UTF-8') as file:
    for line in sft:
        file.write(line + '\n')

Data read. Total count:  583
Input count: 583
Expanded target count: 583
Merged data count: 583
Data return. Total count: 583
/home/elicer/ABSA/data/acos/rest16/result_sft_final_test_best_beam1.txt
